 # Lab 17: Natural Language Processing 1
 #### 12/20/2019

In [34]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [35]:
hotels_df = pd.read_csv('7282_1.csv')

hotels_df.head(5)

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [43]:
rank_hotels_df = hotels_df[['reviews.rating', 'reviews.text']].sort_values(by = 'reviews.rating')
rank_hotels_df.describe()

,reviews.rating
count,35050.000000
mean,3.776431
std,1.416195
min,0.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,10.000000


In [44]:
rank_hotels_df.shape

(35912, 2)

In [37]:
# Check examples of what rating 0.0 means
rank_hotels_df[rank_hotels_df['reviews.rating']== 0.0]

,reviews.rating,reviews.text
35911,0.0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
29873,0.0,to share your opinion of this businesswith YP ...
29874,0.0,to share your opinion of this businesswith YP ...
20524,0.0,to share your opinion of this businesswith YP ...
10802,0.0,to share your opinion of this businesswith YP ...
...,...,...
13409,0.0,to share your opinion of this businesswith YP ...
21713,0.0,to share your opinion of this businesswith YP ...
97,0.0,to share your opinion of this businesswith YP ...
35651,0.0,to share your opinion of this businesswith YP ...


In [60]:
unique_ratings = rank_hotels_df['reviews.rating'].unique()
unique_ratings

array([ 0. ,  1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  3.1,
        3.2,  3.3,  3.4,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.4,  5.8,  6. ,  6.3,  6.7,
        7. ,  7.1,  7.5,  7.9,  8.3,  8.8,  9.2,  9.5,  9.6, 10. ,  nan])

### To Clean Data:
1. remove all data with rating 0 from dataset
2. remove all data without rating from dataset
3. dataset has two rating system in it. one is from 1 to 5, another is 1.0 to 10.0. There is less data with system from 1.0 to 10.0 so I will need to cut some of the data to make data more sutable for sentiment-classifier model.
4. Everything that has rating more that 9.0 will be considered as positeve(2). Everything that has rating 5 - positive. Everything between 5 and 9.0 wil be removed from dataset
5. Everything that has rating less than 3.0 will be considered as negative (0). Everything that has rating 3 or less - negative. All floats number from 3.0 to 9.0 will be removed
6. All reviews wth rating "4" will be considered as neutral (1)




#### Remove all data with rating 0 from dataset

In [61]:
hotels_df_without0 = rank_hotels_df[rank_hotels_df['reviews.rating']!= 0.0]
hotels_df_without0['reviews.rating'].unique()

array([ 1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,  4.5,
        4.6,  4.7,  4.8,  4.9,  5. ,  5.4,  5.8,  6. ,  6.3,  6.7,  7. ,
        7.1,  7.5,  7.9,  8.3,  8.8,  9.2,  9.5,  9.6, 10. ,  nan])

#### Remove all data without rating from dataset

In [62]:
hotels_df_without_nan = hotels_df_without0.dropna()
hotels_df_without_nan['reviews.rating'].unique()

array([ 1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,  4.5,
        4.6,  4.7,  4.8,  4.9,  5. ,  5.4,  5.8,  6. ,  6.3,  6.7,  7. ,
        7.1,  7.5,  7.9,  8.3,  8.8,  9.2,  9.5,  9.6, 10. ])

#### Remove data with ceratin ratings

In [67]:
arr_ratings_exclude = [ 3.1,  3.2, 3.3,  3.4,  3.7,  3.8,  3.9, 4.1,  4.2,  4.3,  4.4,  4.5,
        4.6,  4.7,  4.8,  4.9, 5.4,  5.8,  6. ,  6.3,  6.7,  7., 7.1,  7.5,  7.9,  8.3,  8.8]

hotels_df_clear = hotels_df_without_nan
for el in arr_ratings:
    hotels_df_clear = hotels_df_clear[hotels_df_clear['reviews.rating']!= el]
    
hotels_df_clear['reviews.rating'].unique()
    

array([ 1. ,  1.3,  1.5,  2. ,  2.4,  2.5,  2.8,  2.9,  3. ,  4. ,  5. ,
        9.2,  9.5,  9.6, 10. ])

In [68]:
hotels_df_clear.shape

(34374, 2)

##### Original dataset had 35912 rows so after data cleaning we have 34374 rows which means that we lost 1538 rows of data or 4.28% of original data.

In [70]:
def create_sentiment(num):
    """function to assign 0(negative), 1(neutral), 2(positive) based on ratings number"""
    if num <= 3:
        return 0
    if num == 4:
        return 1
    if num >= 5:
        return 2

In [75]:
hotels_df_clear['sentiment'] = hotels_df_clear['reviews.rating'].apply(create_sentiment)
hotels_df_clear.sample(20)

,reviews.rating,reviews.text,sentiment
22910,2.0,"Very basic and in need of renovation, but pric...",0
19376,5.0,My best friend and I stayed at the hotel for 1...,2
31680,5.0,"Everything I expected plus. Comfy beds, budget...",2
17867,4.0,Almost like a BB in terms of ambiance. Locatio...,1
28063,4.0,I stayed at the hotel for a business trip. The...,1
30990,4.0,It was a little uncomfortable the first room t...,1
11652,5.0,Everything was wonderful. The manager was awes...,2
19888,5.0,My husband and I had a wonderful evening at th...,2
29981,2.0,My expectations were not high when we booked t...,0
25435,5.0,"Beautiful rooms, beautiful property, beautiful...",2
